In [1]:
pwd

'/Users/kelly/metis_v3/Project_5'

In [66]:
book_df = pd.read_csv('books.csv')
ratings_df = pd.read_csv('ratings.csv')
booktags_df = pd.read_csv('booktags.csv')
tags_df = pd.read_csv('tags.csv')

In [8]:
import numpy as np
import pandas as pd

In [17]:
from sklearn.neighbors import NearestNeighbors

In [13]:
from pymongo import MongoClient
client = MongoClient()
db = client.goodreads

In [4]:
import pickle

with open('U.pickle','rb') as read_file:
    U = pickle.load(read_file)

In [26]:
with open('book_vecs_df_drop_na.pickle','rb') as read_file:
    book_vecs_df = pickle.load(read_file)

In [27]:
with open('book_vecs_drop_title.pickle','rb') as read_file:
    book_vecs = pickle.load(read_file)

# Import these functions when conver to .py files!!!!!!

**Collaborative Filtering**

In [315]:
def find_book_rating(ID):
    for book in db.books.find({'book_id':ID}):
        print (book['average_rating'])

def find_book_title(ID):
    for book in db.books.find({'book_id':ID}):
        print(book['original_title'])
        return (book['original_title'])

In [331]:
def find_book_title_noprint(ID):
    for book in db.books.find({'book_id':ID}):
        #print(book['original_title'])
        return (book['original_title'])

In [313]:
def get_recommendations_for_user(userID, U, VT, num_recom):
    recs = []
    for item in range(VT.T.shape[0]):
        recs.append([item+1,np.dot(U[userID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

**Content-Based Filtering**

In [20]:
def get_title_index(book_title):
    try:
        filtered_vecs = book_vecs_df[book_vecs_df.Title == book_title]
        vec_index = filtered_vecs.index[0]
    except:
        print("Book currently checked out, please pick another")
    return vec_index

In [454]:
def get_reading_list(last_book_read):
    
    vec_index = get_title_index(last_book_read)
    
    nn = NearestNeighbors(n_neighbors=1000, metric='cosine', algorithm='brute')
    nn.fit(book_vecs)
    book_recs = nn.kneighbors(np.array([book_vecs.iloc[vec_index]]))
    
    reading_list = book_recs[1][0]
    
    reading_list_recs = []
    
    '''
    from collections import defaultdict

    rec_dict = defaultdict(list) #initializes default dictionary for book recommendations

    
    for ix, book_ix in enumerate(reading_list):
        #print (ix, book_ix)
        rec_dict[last_book_read].append(book_vecs_df.iloc[book_ix].Title)
        
    return rec_dict   
    '''
    
    for book in reading_list:
        if book != '':
            reading_list_recs.append(book_vecs_df.iloc[book].Title)
        else:
            continue
        
    return reading_list_recs

In [494]:
USERID=1
num_recom = 10
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
    find_book_title(item[0])
    find_book_rating(item[0])

The Hunger Games
4.34
Harry Potter and the Philosopher's Stone
4.44
Twilight
3.57
To Kill a Mockingbird
4.25
The Great Gatsby
3.89
The Fault in Our Stars
4.26
The Hobbit or There and Back Again
4.25
The Catcher in the Rye
3.79
Angels & Demons
3.85
Pride and Prejudice
4.24


In [30]:
get_reading_list('The Hunger Games')

defaultdict(list,
            {'The Hunger Games': ['Wideacre',
              'Free Four: Tobias Tells the Divergent Story',
              'The Baller: A Down and Dirty Football Novel',
              '',
              'Playing for Keeps',
              'Sweetheart',
              'Civil War: A Marvel Comics Event',
              'In Too Deep (The 39 Clues, Book 6)',
              'The Help',
              'The Other Side of Dawn']})

**Hybrid Filtering**

In [32]:
'''
Combine the two methods above.  Recommend books by user ratings 
and then filter down recommendation listing based on related content.
'''

'\nCombine the two methods above.  Recommend books by user ratings \nand then filter down recommendation listing based on related content.\n'

In [492]:
def find_book_id(input_title):
    for book in db.books.find({'original_title':input_title}):
        print (book['book_id'])

In [509]:
def find_goodreads_id(input_title):
    for book in db.books.find({'original_title':input_title}):
        print (book['goodreads_book_id'])
        return (book['goodreads_book_id'])

In [544]:
def find_goodreads_id_np(input_title):
    for book in db.books.find({'original_title':input_title}):
        #print (book['goodreads_book_id'])
        return (book['goodreads_book_id'])

In [495]:
#input_title = "Winnie-the-Pooh"
input_title = "Freakonomics: A Rogue Economist Explores the Hidden Side of Everything"
find_book_id(input_title)

92


In [47]:
db.books.find_one()

{'_id': ObjectId('5b88855b21c3f05eeaf945eb'),
 'book_id': 1,
 'goodreads_book_id': 2767052,
 'best_book_id': 2767052,
 'work_id': 2792775,
 'books_count': 272,
 'isbn': 439023483,
 'isbn13': 9780439023480.0,
 'authors': 'Suzanne Collins',
 'original_publication_year': 2008.0,
 'original_title': 'The Hunger Games',
 'title': 'The Hunger Games (The Hunger Games, #1)',
 'language_code': 'eng',
 'average_rating': 4.34,
 'ratings_count': 4780653,
 'work_ratings_count': 4942365,
 'work_text_reviews_count': 155254,
 'ratings_1': 66715,
 'ratings_2': 127936,
 'ratings_3': 560092,
 'ratings_4': 1481305,
 'ratings_5': 2706317,
 'image_url': 'https://images.gr-assets.com/books/1447303603m/2767052.jpg',
 'small_image_url': 'https://images.gr-assets.com/books/1447303603s/2767052.jpg'}

In [58]:
books_by_user = ratings_df.groupby(by = 'user_id')

In [70]:
ratings_df[ratings_df.user_id == 1][ratings_df[ratings_df.user_id == 1].rating == 5]

,user_id,book_id,rating
0,1,258,5
78,1,1796,5
230489,1,11,5
230490,1,1644,5
230492,1,136,5
230495,1,35,5
230502,1,4,5
230505,1,1521,5
230506,1,70,5
230516,1,2002,5


In [316]:
USERID=1
num_recom = 100
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
    find_book_title(item[0])
    find_book_rating(item[0])

The Hunger Games
4.34
Harry Potter and the Philosopher's Stone
4.44
Twilight
3.57
To Kill a Mockingbird
4.25
The Great Gatsby
3.89
The Fault in Our Stars
4.26
The Hobbit or There and Back Again
4.25
The Catcher in the Rye
3.79
Angels & Demons
3.85
Pride and Prejudice
4.24
The Kite Runner
4.26
Divergent
4.24
Nineteen Eighty-Four
4.14
Animal Farm: A Fairy Story
3.87
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
4.1
Män som hatar kvinnor
4.11
Catching Fire
4.3
Harry Potter and the Prisoner of Azkaban
4.53
The Fellowship of the Ring
4.34
Mockingjay
4.03
Harry Potter and the Order of the Phoenix
4.46
The Lovely Bones
3.77
Harry Potter and the Chamber of Secrets
4.37
Harry Potter and the Goblet of Fire
4.53
Harry Potter and the Deathly Hallows
4.61
The Da Vinci Code
3.79
Harry Potter and the Half-Blood Prince
4.54
Lord of the Flies
3.64
An Excellent conceited Tragedie of Romeo and Juliet
3.73
Gone Girl
4.03
The Help
4.45
Of Mice and Men
3.84
Memoirs of a Geisha
4.08
Fifty Sha

In [ ]:
Freakonomics: A Rogue Economist Explores the Hidden Side of Everything

In [108]:
#People who rated Freakonomics a 5
ratings_df[ratings_df.book_id == 92][ratings_df[ratings_df.book_id == 92].rating == 5]

,user_id,book_id,rating
2188,135,92,5
6381,324,92,5
8249,389,92,5
11687,521,92,5
13361,493,92,5
15808,654,92,5
16208,670,92,5
16650,688,92,5
22123,900,92,5
25888,532,92,5


In [103]:
#People who rated Winnie the Pooh a 5
ratings_df[ratings_df.book_id == 444][ratings_df[ratings_df.book_id == 444].rating == 5]

,user_id,book_id,rating
31541,1175,444,5
36202,812,444,5
58134,1794,444,5
74461,2125,444,5
94409,2831,444,5
115607,3221,444,5
159767,1931,444,5
162622,4224,444,5
176418,4567,444,5
180295,3143,444,5


In [279]:
original_rec_list = []

In [381]:
def create_rec_list(user_id):
    original_recs = []
    for item in get_recommendations_for_user(user_id, U, VT, 1000):
        original_recs.append(find_book_title_noprint(item[0]))
    return original_recs

In [345]:
create_rec_list(60)

['The Glass Castle',
 'Angels & Demons',
 'A Thousand Splendid Suns',
 'Fifty Shades of Grey',
 'Animal Farm: A Fairy Story',
 'The Notebook',
 'Memoirs of a Geisha',
 'The Picture of Dorian Gray',
 'Harry Potter and the Half-Blood Prince',
 'Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944',
 'The Girl on the Train',
 'The Fault in Our Stars',
 'The Kite Runner',
 'A Clockwork Orange',
 'Flowers for Algernon',
 'The Perks of Being a Wallflower',
 'The Lost Hero',
 'Speak',
 'If I Stay',
 'Dark Places',
 '',
 'The Bell Jar',
 'A Discovery of Witches',
 "The Titan's Curse",
 'The Five People You Meet in Heaven',
 'Kiss the Girls',
 'The Graveyard Book',
 'Freedom',
 "A People's History of the United States: 1492 to Present",
 'A Clash of Kings',
 'Uglies',
 'Harry Potter and the Cursed Child, Parts One and Two',
 'The Night Circus',
 'The Dead Zone',
 'Choke',
 'The Phantom Tollbooth',
 '',
 'American Psycho',
 'In Cold Blood',
 'City of Heavenly Fire',
 'The Reptile Room',

In [335]:
book_title = "Freakonomics: A Rogue Economist Explores the Hidden Side of Everything"
get_reading_list(book_title)

defaultdict(list,
            {'Freakonomics: A Rogue Economist Explores the Hidden Side of Everything': ["Galileo's Daughter",
              '',
              'The Cake Mix Doctor',
              'Sons and Lovers',
              'New Spring',
              'Chasing Fire',
              'Sea Glass',
              'The Body Farm',
              'The Empty Chair',
              'Five Quarters of the Orange',
              'Animal Dreams',
              'Washington Square',
              'The Devil Wears Prada',
              'The Restaurant at the End of the Universe',
              'Club Dead',
              'Don Quijote de La Mancha',
              'Pebble in the Sky',
              'Down Under',
              'Beautiful Day',
              'Port Mortuary',
              'Memorial do Convento',
              'Unnatural Exposure',
              'The Sands of Time',
              'A History of Western Philosophy And Its Connection with Political and Social Circumstances from the Earliest

In [ ]:
#filter original rec list based on related content of last book read (rated)

'''

This function finds the user ID
Takes a book highly rated by the user (if not 5, 4, if not 4, 3, etc.)
not already read by the user
Takes the rec list from the user ID, based on ratings of other users

filters that rec list based on related content of the randomly selected highly rated book
returns top 5
'''

In [212]:
for rec in original_rec_list:
    if rec in get_reading_list(book_title):
        print (rec, "YAY")
    else:
        print (rec, "NO!")

Breaking Dawn NO!
The Notebook NO!
An Excellent conceited Tragedie of Romeo and Juliet NO!
Water for Elephants NO!
The Hobbit or There and Back Again NO!
City of Bones NO!
Memoirs of a Geisha NO!
The Road NO!
Jurassic Park NO!
A Game of Thrones NO!
The Secret Garden NO!
Angels & Demons NO!
Le Petit Prince NO!
The Tragicall Historie of Hamlet, Prince of Denmark NO!
The Great Gatsby NO!
Män som hatar kvinnor NO!
Deception Point NO!
Steve Jobs NO!
The Kite Runner NO!
The Handmaid's Tale NO!
Jane Eyre NO!
Moby Dick; or, The Whale NO!
The Pillars of the Earth NO!
The Perks of Being a Wallflower NO!
Atonement NO!
Gone Girl NO!
The Voyage of the Dawn Treader NO!
The Silence of the Lambs NO!
Catch-22 NO!
The Host NO!
Slaughterhouse-Five, or The Children's Crusade: A Duty-Dance with Death  NO!
Harry Potter and the Half-Blood Prince NO!
Reflected in You NO!
Charlotte's Web NO!
Snow Flower and the Secret Fan NO!
Artemis Fowl NO!
Sense and Sensibility NO!
The Picture of Dorian Gray NO!
Watership D

En man som heter Ove NO!
What Alice Forgot NO!
Two-Way Street NO!
Panic NO!
Among the Betrayed NO!
Peace Like a River NO!
Wool Omnibus NO!
Americanah NO!
るろうに剣心 1 NO!
The Silver Chair NO!
Como agua para chocolate NO!
The Great Divorce NO!
What If?: Serious Scientific Answers to Absurd Hypothetical Questions NO!
The Unbecoming of Mara Dyer NO!
The Edge of Never NO!
Dracula NO!
A Little Princess NO!
I'll Give You the Sun NO!
We Need to Talk About Kevin NO!
Enchanters' End Game NO!
The Constant Princess NO!
The Paper Magician NO!
While I Was Gone NO!
Safe Haven NO!
"M" is for Malice NO!
The Restaurant at the End of the Universe NO!
The Dark Is Rising NO!
We Were Liars NO!
The Bourne Supremacy NO!
No, David! NO!
Black Water NO!
Faefever NO!
An Ember in the Ashes NO!
Walk Two Moons NO!
Paradise Lost NO!
 NO!
Fragile Things: Short Fictions and Wonders NO!
The Boston Girl NO!
Rosemary's Baby NO!
Childhood's End NO!
In the Woods NO!
The Lost Wife NO!
Anansi Boys NO!
Sphere NO!
The Partly Cloud

Ringworld NO!
La Vérité sur l'affaire Harry Quebert NO!
Bloodfever NO!
Hell's Angels NO!
The Memory Keeper's Daughter NO!
The 3 Mistakes of My Life NO!
Lucky NO!
I Am America (And So Can You!) NO!
Eat, pray, love: one woman's search for everything across Italy, India and Indonesia NO!
The Seeress of Kell NO!
The Diviners NO!
Stone of Tears NO!
The Magus NO!
道德經 [dào dé jīng] NO!
Vanishing Acts NO!
Atlas Shrugged NO!
Revelations NO!
The Celestine Prophecy NO!
Hija de la fortuna NO!
Lean Mean Thirteen NO!
Kiss of a Demon King NO!
Lilac Girls NO!
Hawaii NO!
Good as Gone NO!
The Rescue NO!
Just So Stories for Little Children NO!
The Amityville Horror NO!
Flyte NO!
An Echo in the Bone NO!
Where Rainbows End NO!
The River of Doubt: Theodore Roosevelt's Darkest Journey NO!
Beautiful Stranger NO!
Reaper's Property NO!
Memories of Ice NO!
Panda Bear, Panda Bear, What Do You See? NO!
Wolf Brother NO!
The Disappearing Spoon: And Other True Tales of Madness, Love, and the History of the World from

The Black Dagger Brotherhood: An Insider's Guide NO!
A Wind in the Door NO!
Goddess NO!
The White Queen NO!
Heir to the Empire NO!
Narcissus in Chains (Anita Blake, Vampire Hunter, #10) NO!
The End of Eternity NO!
Smoke Gets in Your Eyes: And Other Lessons from the Crematory NO!
三体 NO!
Låt den rätte komma in NO!
The Little Friend NO!
The Hero of Ages NO!
Neuromancer NO!
Lost in a Good Book NO!
Just One Look NO!
Key of Valor (Key Trilogy, #3) NO!
Mister Pip NO!
The Search NO!
Leven en werken van de Kabouter NO!
The Kingmaker's Daughter NO!
The Story of Beautiful Girl NO!
Love Anthony NO!
Mythologies NO!
Ramona Forever NO!
Rise of Empire NO!
Brown Girl Dreaming NO!
When We Were Orphans NO!
The Stonekeeper's Curse NO!
Fang NO!
بيكاسو وستاربكس NO!
Freedom™ NO!
Us NO!
Complications: A Surgeon's Notes on an Imperfect Science NO!
of course i love you NO!
Half the Sky: Turning Oppression into Opportunity for Women Worldwide NO!
The Descendants NO!
Ensaio Sobre a Cegueira NO!
Foundation's Edge 

In [474]:
#find books user has previously rated

def previously_read(user_num):

    userread_df = ratings_df[ratings_df.user_id == user_num].join(
    book_df, how = 'left', lsuffix = ' ', on = 'book_id')[['user_id','book_id','original_title']]
    
    previously_readbooks = list(userread_df.original_title)
    
    return previously_readbooks

In [262]:
#find books user has rated a 5

def find_favorites(user_num):

    userread_df = ratings_df[ratings_df.user_id == user_num][ratings_df[ratings_df.user_id == user_num].rating == 5].join(
    book_df, how = 'left', lsuffix = ' ', on = 'book_id')[['user_id','book_id','original_title']]
    
    fav_books = list(userread_df.original_title)
    
    return fav_books

In [362]:
find_favorites(9143)

['A Time to Kill',
 'The Book Thief',
 'Water for Elephants',
 'Twilight',
 'The Great Gatsby',
 'Fight Club',
 'A Thousand Splendid Suns',
 'Guns, Germs, and Steel: The Fates of Human Societies',
 'Where the Wild Things Are',
 'Eldest',
 'Daughter of Smoke & Bone',
 'Chronicles of The Black Company (The Black Company / Shadows Linger / The White Rose)',
 'Harry Potter and the Half-Blood Prince',
 'City of Bones',
 'Dark Places',
 "The Handmaid's Tale",
 'The Thirteenth Tale',
 'The Night Circus',
 'Abhorsen',
 'The Five People You Meet in Heaven',
 'A Discovery of Witches',
 "Angela's Ashes: A Memoir",
 'World War Z: An Oral History of the Zombie War',
 "Seriously...I'm Kidding",
 'Of Mice and Men ',
 'The Last Letter From Your Lover',
 'Animal, Vegetable, Miracle: A Year of Food Life',
 'Unbroken: A World War II Story of Survival, Resilience, and Redemption',
 'The Cat in the Hat',
 'Ender in Exile (The Ender Quintet, #2)',
 'Pet Sematary',
 'Clockwork Princess',
 'Before I Go to Sle

In [565]:
#np.random.choice(find_favorites(9143))

In [562]:
#Find people who rated Freakonomics a 5
#book_num = 92
#ratings_df[ratings_df.book_id == book_num][ratings_df[ratings_df.book_id == book_num].rating == 5]

In [550]:
def find_bookurl(book_title):
    goodreads_book_id = find_goodreads_id_np(book_title)
    return 'https://www.goodreads.com/book/show/'+str(goodreads_book_id)

In [566]:
def hybrid_filtering(user_id):
    
    last_book_read = np.random.choice(find_favorites(user_id)) #picks favorite book from user's previously read books
    print('Last Book Read:', last_book_read)
    
    already_read_list = previously_read(user_id)   
    original_list = create_rec_list(user_id) #takes in user id and returns original rec list
    reading_list = get_reading_list(last_book_read) #returns reading list
    
    book_recs = []
    
    for book in original_list:
        if book in reading_list == '':
            pass
        elif book not in already_read_list:
            book_recs.append(book)
    
    from collections import defaultdict
    url_dict = defaultdict(list)
    
    #url_dict = {}
    for ix, rec in enumerate(book_recs[:10]):
        book_link = find_bookurl(rec)
        url_dict[ix] = book_link
    
    print('Book Recommendations:', book_recs[:10])
    #print(url_dict)

In [567]:
#Find people who rated Freakonomics a 5
#book_num = 3
#ratings_df[ratings_df.book_id == book_num][ratings_df[ratings_df.book_id == book_num].rating == 5]

In [568]:
hybrid_filtering(2575)

Last Book Read: City of Bones
Book Recommendations: ['Allegiant', 'Of Mice and Men', "Marley & Me: Life and Love with the World's Worst Dog", 'Anne of Green Gables', 'Water for Elephants', 'A Time to Kill', 'The Adventures of Huckleberry Finn', 'Middlesex', 'Animal Farm: A Fairy Story', 'The Book Thief']


# Make hybrid filtering dynamic for new users!!!!!